# Testing the Prediction Model Pipeline
This notebook tests the `ModelPredictor` class which uses predefined system prompts to generate answers for different scenarios. Below are the steps to use this notebook:

## Setup
First, make sure that all required libraries are installed and import the necessary modules. You will also need to ensure that the prediction model and system prompts scripts are available as modules.

In [1]:
import json
from prediction_model import ModelPredictor
from system_prompts import SystemPrompts
from datetime import datetime
prompts_manager = SystemPrompts()


## Load Data
Load the data which contains various scenarios and questions that will be processed by the prediction model.

In [2]:
with open('../data/processed/extracted_data.json', 'r') as file:
    files = json.load(file)

for data in files:
    print(data)

{'file': '202021-2-g1-midterm-aisya.pdf', 'scenario': 'Aisya, 38, is a newly-minted equity partner specialising in commercial law at a mid-sized\nSingapore law ﬁrm. Her husband Kwan, also 38, does investments and trading at a local\nbank. The two had recently gotten married in Jan 2020.\nOn 1 Feb 2020, they moved into a 3 storey landed bungalow they had bought in the Katong\narea. The house is one of those nice heritage houses with a long colonial history. It\nwas designed and built in 1940 by Due South Ltd, now a big listed Singapore property\ndeveloper. Aisya and Kwan (henceforth “A&K”) are the ﬁfth owners of the house. They\nofﬁcially took over ownership on 31 Oct 2019.\nBefore moving in, they engaged Safe Interiors Pte Ltd (“SI”), an interior design and furni-\nture company, to plan and also implement a series of renovations. This included building\na new wall fountain in their living room (the sort where water ﬂows down along a fea-\nture wall and into a pool). To preserve the her

## 

## Setup Pipeline

In [3]:
# Retrieve and print a system prompt using the SystemPrompts class
prompt_key = 'ans_tort_qns'
sys_prompt = prompts_manager.retrieve(prompt_key)
print(sys_prompt)

Background: I am a Singapore lawyer specialized in the Law of Torts.

Task: Analyze provided hypothetical scenarios under Singapore Tort Law, addressing specific questions with detailed legal principles and relevant case law.

Instructions:

1. Read and understand the scenario provided.
2. Identify all legal issues related to the question.
3. Organize the answer using the IRAC (Issue, Rule, Application, Conclusion) structure:
a. **Issue**: Clearly state the legal issue(s) identified.
b. **Rule**: Define and explain the relevant legal principles, statutes, and case law that apply to the issue.
c. **Application**: Apply the legal principles to the specific facts of the scenario, analyzing how the rule affects the case.
d. **Conclusion**: Provide a reasoned conclusion based on the application of the rule to the facts.
4. Justify answers with references to relevant statutes and cases.
5. Clearly state any assumptions and their implications.
6. Organize the answer clearly, using headings or

In [4]:
# Load settings from settings.json
with open('settings.json', 'r') as settings_file:
    settings = json.load(settings_file)

model_name = settings["model"]
# model_name = 'gpt-4-turbo'
print(model_name)

predictor = ModelPredictor(
    system_prompt=sys_prompt,
    model_name = model_name
)

llama-3


c:\Users\benhz\Documents\GitHub\SMU-LLM-Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.85s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Generate Answers
This function will use the `ModelPredictor` to generate answers for the loaded questions multiple times, based on the epochs specified.

In [5]:
def generate_answers(data, epoch):

    # Iterate over each file in the JSON data
    scenario = data['scenario']
    questions = data['questions']
    answers = {}

    # For each question, prepare the prompt and get the model's prediction multiple times
    for i, question in enumerate(questions):
        question_key = f"question_{i + 1}"
        answers[question_key] = []

        for _ in range(epoch):
            prompt = f"Scenario: {scenario}\nQuestion: {question}\n\nMy answer is:"
            answer = predictor.predict(prompt)
            print(answer)
            answers[question_key].append(answer)

    # Add the answers to the data
    data['answers'] = answers

    return data

## Run Prediction
Execute the prediction process and save the updated data with the generated answers.

In [6]:
for data in files:
    updated_data = generate_answers(data, epoch=3)

    name = data['file'].split(".")[0]

    output_filename = f'../results/{name}_{model_name}.json'
    with open(output_filename, 'w') as file:
        json.dump(updated_data, file, indent=4)

    print(f"Processing complete. The updated data is saved in '{output_filename}'.")

c:\Users\benhz\Documents\GitHub\SMU-LLM-Project\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:566.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[{'generated_text': "Background: I am a Singapore lawyer specialized in the Law of Torts.\n\nTask: Analyze provided hypothetical scenarios under Singapore Tort Law, addressing specific questions with detailed legal principles and relevant case law.\n\nInstructions:\n\n1. Read and understand the scenario provided.\n2. Identify all legal issues related to the question.\n3. Organize the answer using the IRAC (Issue, Rule, Application, Conclusion) structure:\na. **Issue**: Clearly state the legal issue(s) identified.\nb. **Rule**: Define and explain the relevant legal principles, statutes, and case law that apply to the issue.\nc. **Application**: Apply the legal principles to the specific facts of the scenario, analyzing how the rule affects the case.\nd. **Conclusion**: Provide a reasoned conclusion based on the application of the rule to the facts.\n4. Justify answers with references to relevant statutes and cases.\n5. Clearly state any assumptions and their implications.\n6. Organize t

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


[{'generated_text': "Background: I am a Singapore lawyer specialized in the Law of Torts.\n\nTask: Analyze provided hypothetical scenarios under Singapore Tort Law, addressing specific questions with detailed legal principles and relevant case law.\n\nInstructions:\n\n1. Read and understand the scenario provided.\n2. Identify all legal issues related to the question.\n3. Organize the answer using the IRAC (Issue, Rule, Application, Conclusion) structure:\na. **Issue**: Clearly state the legal issue(s) identified.\nb. **Rule**: Define and explain the relevant legal principles, statutes, and case law that apply to the issue.\nc. **Application**: Apply the legal principles to the specific facts of the scenario, analyzing how the rule affects the case.\nd. **Conclusion**: Provide a reasoned conclusion based on the application of the rule to the facts.\n4. Justify answers with references to relevant statutes and cases.\n5. Clearly state any assumptions and their implications.\n6. Organize t

In [10]:
output_filename = f'../results/exam_answerss.json'
updated_data = {
  "pipeline": "openai", 
  "model": "gpt-4-turbo"
}
with open(output_filename, 'w') as file:
    json.dump(updated_data, file, indent=4)

## Manage System Prompt

In [ ]:
# Store a new prompt 
prompt_key = 'test_123'
new_prompt_text = """
testing1232412432
"""
prompts_manager.store(prompt_key, new_prompt_text)
print('New prompt stored successfully.')

In [ ]:
# Delete a prompt
prompt_key = 'test_123'

prompts_manager.remove(prompt_key)
print('Prompt removed successfully.')